In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import selenium
import urllib.request

In [2]:
df = pd.read_csv("C:/worksplaces/workspace_project/project_visual/restaurant_data/data/restaurant.csv", encoding="utf-8", index_col=0)
df

,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y
0,북악파크부페,영업/정상,110846.0,서울특별시 종로구 평창동 110-1번지,서울특별시 종로구 평창길 352 (평창동),197993.205677,456584.627065
1,시민식당 2호점,영업/정상,110290.0,서울특별시 종로구 인사동 98번지,"서울특별시 종로구 종로11길 10, 1,2층 (인사동)",198681.571899,452050.274345
2,일번우동,영업/정상,110126.0,서울특별시 종로구 종로6가 129-1번지,서울특별시 종로구 종로41길 10 (종로6가),200533.453584,452156.900283
3,서촌소국밥,영업/정상,110043.0,서울특별시 종로구 통인동 27-2,서울특별시 종로구 필운대로6길 17-1 (통인동),197250.385438,453135.451266
4,빽다방 동대문역점,영업/정상,110843.0,서울특별시 종로구 창신동 696-1,서울특별시 종로구 종로 289 (창신동),200811.925354,452162.569488
...,...,...,...,...,...,...,...
124905,20bucks,영업/정상,130831.0,서울특별시 동대문구 이문동 316-2 지하1층,"서울특별시 동대문구 휘경로2길 19, 지하1층 (이문동)",205294.170617,454765.261112
124906,DNJ,영업/정상,139230.0,서울특별시 노원구 하계동 256-11 보스턴 산부인과,"서울특별시 노원구 노원로 244, 보스턴 산부인과 1층 (하계동)",206490.594308,459688.060943
124907,희래등반점,영업/정상,110811.0,서울특별시 종로구 명륜3가 1-14,서울특별시 종로구 명륜길 31 (명륜3가),199506.183606,454029.397330
124908,별달맛집,영업/정상,135894.0,서울특별시 강남구 신사동 609-1 지하1층,"서울특별시 강남구 압구정로30길 23 (신사동,지하1층)",202534.550000,447079.205000


In [3]:
df.isnull().sum()

BPLCNM            1
TRDSTATENM        0
SITEPOSTNO       80
SITEWHLADDR      80
RDNWHLADDR     1660
X              1880
Y              1880
dtype: int64

### Null 값 처리 어떻게 할지 고민해보자
- BPLCNM 없는 로우의 주소를 검색해서 입력해주자
- SITEPOSTNO 칼럼은 필요없음 
- SITEWHLADDR에서 없는 데이터는 RDNWHLADDR 데이터에 있을까 (또는 반대)
- X와 Y에 없는 데이터의 위경도 가져오는 법?
- X와 Y가 없는 데이터의 주소는 다 있는 건가?

In [4]:
nm_null = df[df['BPLCNM'].isnull()].index

df.iloc[nm_null, 0] = ('엔에이')
df.iloc[nm_null]

,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y
10449,엔에이,영업/정상,100848.0,서울특별시 중구 을지로4가 35번지 2층,"서울특별시 중구 창경궁로5길 27, 2층 (을지로4가)",199614.974362,451625.623159


In [5]:
df[(df['SITEWHLADDR'].isnull()) & (df['RDNWHLADDR'].isnull())]

,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y


> df['SITEWHLADDR'] 이 null이면서 df['RDNWHLADDR']이 null 인 값은 없음

In [6]:
addr_null = df[df['SITEWHLADDR'].isnull()].index

In [7]:
df['SITEWHLADDR'] = np.where(pd.notnull(df['SITEWHLADDR']) == True, df['SITEWHLADDR'], df['RDNWHLADDR'])

In [8]:
df.isnull().sum()

BPLCNM            0
TRDSTATENM        0
SITEPOSTNO       80
SITEWHLADDR       0
RDNWHLADDR     1660
X              1880
Y              1880
dtype: int64

In [9]:
df['BPLCNM'].replace('메가엠지씨커피', '메가커피', inplace=True)

In [11]:
df['BPLCNM'].replace('꾸쁘(Coupe)', '라꾸쁘', inplace=True)
df['BPLCNM'].replace('메가커피 우장산점', '메가커피 우장산역점', inplace=True)


### 1

In [12]:
craw_data = df[['BPLCNM', 'SITEWHLADDR', 'RDNWHLADDR']]

In [13]:
craw_data['Gu'] = craw_data.SITEWHLADDR.str.split(' ').str[1]
craw_data['Dong'] = craw_data.SITEWHLADDR.str.split(' ').str[2]

C:\Users\wogml\AppData\Local\Temp/ipykernel_10640/739233450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  craw_data['Gu'] = craw_data.SITEWHLADDR.str.split(' ').str[1]
C:\Users\wogml\AppData\Local\Temp/ipykernel_10640/739233450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  craw_data['Dong'] = craw_data.SITEWHLADDR.str.split(' ').str[2]


In [14]:
craw_data = craw_data[['Gu', 'Dong', 'BPLCNM']]
address = craw_data['Gu'] + " " + craw_data['Dong'] + " " + craw_data['BPLCNM']

In [16]:
craw_list = list(craw_data['BPLCNM'].unique())

In [17]:
address_list = list(address)
#print(address_list)

In [19]:
row_data = {'address' : address_list}
rest_add = pd.DataFrame(row_data)

# 크롤링


selenium<br>
https://0433.tistory.com/41

In [21]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from tqdm import tqdm

**selenium test**

In [22]:
target_url = 'https://search.naver.com/search.naver?sm=tab_sly.hst&where=nexearch&query=종로구%20통인동%20서촌소국밥'

driver = webdriver.Chrome('C:/worksplaces/workspace_project/project_visual/drivers/chromedriver.exe')

driver.get(target_url)
sleep(1)
#driver.find_element_by_xpath('//*[@id="_title"]/a').click()

C:\Users\wogml\AppData\Local\Temp/ipykernel_10640/1800054280.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/worksplaces/workspace_project/project_visual/drivers/chromedriver.exe')


**beautifulsoup test**

내일 하자 !!! 
메뉴 나누기

In [23]:
target_url = 'https://search.naver.com/search.naver?sm=tab_sly.hst&where=nexearch&query=종로구+창신동+실비집'

result = []
tmp = dict()

resp = requests.get(target_url)
soup = BeautifulSoup(resp.text, 'html.parser')

tmp['address'] = address_list[3]

if soup.find('span', {'class':'_3XamX'}):
    bistro_nm = soup.find('span', {'class':'_3XamX'}).text
    bistro_fd = soup.find('span', {'class':'_3ocDE'}).text
    bistro_tel = soup.find('span', {'class':'_3ZA0S'}).text

    tmp['storename'] = bistro_nm
    tmp['storetype'] = bistro_fd
    tmp['telephone'] = bistro_tel
    
elif soup.find('span', {'class':'place_bluelink _1RlVL'}):
    if len(soup.find_all('mark')) <= 2:
        
        bistro_nm1 = soup.find_all('mark')
        tmp['storename'] = bistro_nm1[0].text + bistro_nm1[1].text
        
    
    elif soup.find('mark'):
        bistro_nm2 = soup.find('mark')
        tmp['storename'] = bistro_nm2.text
    
    else:
        pass
    
    bistro_fd = soup.find('span', {'class':'_1tj6W'}).text
    tmp['storetype'] = bistro_fd

        
    

if soup.find('ul', {'class':'_1WK4L _2EEOB'}):
    
    menu_div1 = soup.find('ul', {'class':'_1WK4L _2EEOB'})
    menu_list1 = menu_div1.select('li')

    menu_lst1 = []

    for chd1 in menu_list1:

        menu_nm1 = chd1.find('a').string
        price1 = chd1.find('div', {'class':'_18uV7'}).text

        menu_lst1.append(menu_nm1 + " " + price1)

        tmp['menu'] = menu_lst1
    
    result.append(tmp)

    
elif soup.find('ul', {'class':'V1UmJ Zew_k'}):
    
    menu_div2 = soup.find('ul', {'class':'V1UmJ Zew_k'})
    menu_list2 = menu_div2.select('li')
    
    menu_lst2 = []
    
    for chd2 in menu_list2:
        menu_nm2 = chd2.find('a').text
        price2 = chd2.find('em').text
        
        menu_lst2.append(menu_nm2 + " " + price2)
        
        tmp['menu'] = menu_lst2
        
    result.append(tmp) 
    
elif soup.find('div', {'class':'_1mAZf'}):
    menu_span = soup.find_all('span', {'class':'_1TRlX'})
    
    menu_lst3 = []
    #print(menu_div3)
    
    tmp['telephone'] = menu_span[0].text
    
    menu_lst3.append(menu_span[-2].text)
    menu_lst3.append(menu_span[-1].text)
    
    tmp['menu'] = menu_lst3
    
    
    result.append(tmp)
        
    
        
        
        
print(result)

[{'address': '종로구 통인동 서촌소국밥', 'storename': '실비집', 'storetype': '전,빈대떡', 'telephone': '010-8835-5294', 'menu': ['모듬전(2인) 5,000', '김치찌게(2인) 5,000']}]


In [24]:
test_list = address_list[:100]

In [25]:
len(address_list)

124910

In [26]:
###백업용 

In [27]:
result = []

def open_browser(address_list):
    
    for i in tqdm(range(len(address_list) )):
        
        target_url = 'https://search.naver.com/search.naver?sm=tab_sly.hst&where=nexearch&query='

        url = target_url + address_list[i]
        
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        try:
            # storename
            bistro_nm = soup.find('span', {'class':'_3XamX'}).text
            # storetype
            bistro_fd = soup.find('span', {'class':'_3ocDE'}).text

            tmp = dict()
            tmp['storeaddr'] = address_list[i]
            tmp['storename'] = bistro_nm
            tmp['storetype'] = bistro_fd

        
            # 전화번호 
            try:
                bistro_tel = soup.find('span', {'class':'_3ZA0S'}).text
                tmp['callnum'] = bistro_tel

            except:
                tmp['callnum'] = ' '

                
                
            if soup.find('ul', {'class':'_1WK4L _2EEOB'}):

                try: 
                    menu_div = soup.find('ul', {'class':'_1WK4L _2EEOB'})
                    menu_list = menu_div.select('li')

                    menu_lst = []

                  # 메뉴
                    for chd in menu_list:
                        menu_nm = chd.find('a').text
                        price = chd.find('div', {'class':'_18uV7'}).text

                        menu_lst.append(menu_nm + " " + price)
                        tmp['menu'] = menu_lst

                    result.append(tmp)

                except:
                    pass


            elif soup.find('ul', {'class':'V1UmJ Zew_k'}):
                try:  
                    menu_div2 = soup.find('ul', {'class':'V1UmJ Zew_k'})
                    menu_list2 = menu_div2.select('li')

                    menu_lst2 = []

                    for chd2 in menu_list2:
                        menu_nm2 = chd2.find('a').text
                        price2 = chd2.find('em').text

                        menu_lst2.append(menu_nm2 + " " + price2)

                        tmp['menu'] = menu_lst2

                    result.append(tmp) 

                except:
                    pass
            
            else: 
                pass
            
        
        except: 
            pass
            
        

In [31]:
open_browser(address_list)

100%|███████████████████████████████████████████████████████████████████████| 124910/124910 [19:08:22<00:00,  1.81it/s]


In [379]:
result_df = pd.DataFrame(result)

In [380]:
result_df

,storeaddr,storename,storetype,callnum,menu
0,종로구 통인동 서촌소국밥,서촌소국밥,한식,070-8885-7677,"[소머리국밥 12,000원, 제육볶음 10,000원, 사골만두국 10,000원]"
1,종로구 창신동 빽다방 동대문역점,빽다방 동대문역점,테이크아웃커피,0507-1341-0950,"[아메리카노HOT 1,500원, 아메리카노ICE 2,000원, 더블에스프레소 1,5..."
2,종로구 소격동 주식회사 가배도 삼청,가배도 삼청,카페,02-732-4542,"[말차 라떼 6,500원, 티라미수(일반/말차/얼그레이) 7,000원, 가배 4,9..."
3,종로구 충신동 기생,기생,"우동,소바",,"[우동 12,000원, 고기채소나베 12,000원, 오뎅채소나베 12,000원]"
4,종로구 무악동 상구네백반,상구네집,"백반,가정식",0507-1325-5324,"[코다리찜 소 20,000원, 올갱이해장국 10,000원, 북어콩나물해장국 8,000원]"
...,...,...,...,...,...
42324,광진구 화양동 매드쉐프떡볶이,매드쉐프 떡볶이,떡볶이,0507-1325-2388,"[매드쉐프 떡볶이 7,000원, 오징어튀김 3,600원, 왕새우튀김 4,000원]"
42325,강서구 화곡동 예담골남원추어탕,예담골남원추어탕,추어탕,02-2646-8002,"[추어탕+돌솥 8,000원, 통추어탕+돌솥 9,000원, 우렁추어탕+돌솥 10,00..."
42326,서초구 잠원동 오공복이 논현역점,오공복이 논현역점,한식,0507-1416-6616,"[스테이크덮밥 11,000원, 호르몬동 10,900원, 명란계란버터밥 7,500원,..."
42327,마포구 도화동 옛날불고기 대상,대상 옛날불고기,한식,02-702-3391,"[옛날불고기 13,000원, 뭉티기 육사시미 29,000원, 뭉티기 육회 29,000원]"


#### save

In [381]:
result_df.to_csv("C:/worksplaces/workspace_project/project_visual/restaurant_data/data/menu.csv", index=None)

In [382]:
result[0].get('menu')

['소머리국밥 12,000원', '제육볶음 10,000원', '사골만두국 10,000원']

**menu 리스트를 분리해서 칼럼 저장**

In [384]:
menu_list = result_df['menu']

result_df['menu1'] = menu_list.str.get(0)
result_df['menu2'] = menu_list.str.get(1)
result_df['menu3'] = menu_list.str.get(2)

In [385]:
result_df.drop(['menu'] ,axis=1, inplace=True)

In [386]:
result_df.to_csv("C:/worksplaces/workspace_project/project_visual/restaurant_data/data/menu.csv", index=None)

In [387]:
menu_list1 = result_df['menu1'].str.split(' ')
menu_list2 = result_df['menu2'].str.split(' ')
menu_list3 = result_df['menu3'].str.split(' ')

result_df['menu1_price'] = menu_list1.str.get(-1)
result_df['menu2_price'] = menu_list2.str.get(-1)
result_df['menu3_price'] = menu_list3.str.get(-1)

In [388]:
result_df['menu1_price'] = result_df['menu1_price'].str.replace('원', '')
result_df['menu2_price'] = result_df['menu2_price'].str.replace('원', '')
result_df['menu3_price'] = result_df['menu3_price'].str.replace('원', '')

In [389]:
result_df.fillna("{'menu2_price': 0 , 'menu3_price': 0}", inplace=True)

---

숫자 ~ 숫자 값을 가진 가격데이터를 최소 가격만 넣기 

In [390]:
special1 = result_df[result_df['menu1_price'].str.contains("~")]

special_list1 = special1['menu1_price'].str.split('~')
special1['menu1_price'] = special_list1.str.get(0)

special2 = result_df[result_df['menu2_price'].str.contains("~")]
special_list2 = special2['menu2_price'].str.split('~')
special2['menu2_price'] = special_list2.str.get(0)

special3 = result_df[result_df['menu3_price'].str.contains("~")]
special_list3 = special3['menu3_price'].str.split('~')
special3['menu3_price'] = special_list3.str.get(0)

C:\Users\wogml\AppData\Local\Temp/ipykernel_10640/1366538459.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  special1['menu1_price'] = special_list1.str.get(0)
C:\Users\wogml\AppData\Local\Temp/ipykernel_10640/1366538459.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  special2['menu2_price'] = special_list2.str.get(0)
C:\Users\wogml\AppData\Local\Temp/ipykernel_10640/1366538459.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [391]:
result_df['menu1_price'].iloc[list(special1.index)] = special1['menu1_price']
result_df['menu2_price'].iloc[list(special2.index)] = special2['menu2_price']
result_df['menu3_price'].iloc[list(special3.index)] = special3['menu3_price']

In [392]:
result_df

,storeaddr,storename,storetype,callnum,menu1,menu2,menu3,menu1_price,menu2_price,menu3_price
0,종로구 통인동 서촌소국밥,서촌소국밥,한식,070-8885-7677,"소머리국밥 12,000원","제육볶음 10,000원","사골만두국 10,000원","12,000","10,000","10,000"
1,종로구 창신동 빽다방 동대문역점,빽다방 동대문역점,테이크아웃커피,0507-1341-0950,"아메리카노HOT 1,500원","아메리카노ICE 2,000원","더블에스프레소 1,500원","1,500","2,000","1,500"
2,종로구 소격동 주식회사 가배도 삼청,가배도 삼청,카페,02-732-4542,"말차 라떼 6,500원","티라미수(일반/말차/얼그레이) 7,000원","가배 4,900원","6,500","7,000","4,900"
3,종로구 충신동 기생,기생,"우동,소바",,"우동 12,000원","고기채소나베 12,000원","오뎅채소나베 12,000원","12,000","12,000","12,000"
4,종로구 무악동 상구네백반,상구네집,"백반,가정식",0507-1325-5324,"코다리찜 소 20,000원","올갱이해장국 10,000원","북어콩나물해장국 8,000원","20,000","10,000","8,000"
...,...,...,...,...,...,...,...,...,...,...
42324,광진구 화양동 매드쉐프떡볶이,매드쉐프 떡볶이,떡볶이,0507-1325-2388,"매드쉐프 떡볶이 7,000원","오징어튀김 3,600원","왕새우튀김 4,000원","7,000","3,600","4,000"
42325,강서구 화곡동 예담골남원추어탕,예담골남원추어탕,추어탕,02-2646-8002,"추어탕+돌솥 8,000원","통추어탕+돌솥 9,000원","우렁추어탕+돌솥 10,000원","8,000","9,000","10,000"
42326,서초구 잠원동 오공복이 논현역점,오공복이 논현역점,한식,0507-1416-6616,"스테이크덮밥 11,000원","호르몬동 10,900원","명란계란버터밥 7,500원","11,000","10,900","7,500"
42327,마포구 도화동 옛날불고기 대상,대상 옛날불고기,한식,02-702-3391,"옛날불고기 13,000원","뭉티기 육사시미 29,000원","뭉티기 육회 29,000원","13,000","29,000","29,000"


---
주소랑 지점타이틀을 분리 

In [393]:
result_list = result_df['storeaddr'].str.split(' ')

result_df['addr'] = result_list.str.get(0) + ' ' + result_list.str.get(1)

result_df.drop(['storeaddr'], axis=1, inplace=True)

지점 타이틀 리스트에서 str로 변환

In [394]:
'''
for i in range(len(result_df['title'])):
    s = " ".join(result_df['title'][i])
    result_df['title'][i] = s

'''

'\nfor i in range(len(result_df[\'title\'])):\n    s = " ".join(result_df[\'title\'][i])\n    result_df[\'title\'][i] = s\n\n'

In [395]:
result_df.to_csv("C:/worksplaces/workspace_project/project_visual/restaurant_data/data/menu.csv", index=None)